# Calculating percent proficiency and its percent change in New York State education assessment data

## Getting Payments Data
1. Access yearly payment data from [OpenPaymentsData.cms.gov](https://openpaymentsdata.cms.gov/browse).
2. For each year (2013-2016), View Data -> Filter -> Recipient_State = 'NY'
3. Rename file to 'General_Payment_NY_`year`_.csv'.

## Getting Perscription Data
1. Access Medicare Provider Utilization and Payment Data: Part D Prescriber Summary Table CY`year` from [data.cms.gov/](https://data.cms.gov/).
2. For each year (2013-2016), View Data -> Filter -> Recipient_State = 'NY'
3. Rename file to 'Medicare_Perscriber_Summary_`year`_.csv'.


In [144]:
import agate
import csv
from collections import OrderedDict

Medicare categorizes a physican's opiate-related drugs according to [this methodology](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/Opioid_Methodology.pdf). I used the following [drug names](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/OpioidDrugList.zip) to categorize whether a drug company's payment involved opiate-related drugs. However, [NYS Health Foundation](https://nyshealthfoundation.org/wp-content/uploads/2018/06/following-the-money-pharmaceutical-payments-opioid-prescribing-june-2018.pdf) excluded some drugs from this categorization. The “Excluded” column to the right denotes a few of those drugs that I think should be filtered out according to the study’s methodology, which is "Namely, all buprenorphine drugs, including buprenorphine/naloxone formulations are excluded, as they are used for opioid addiction treatment. In addition, nonsteroidal anti-inflammatory drugs (NSAIDs) are excluded.”

In [145]:
opiates = {}
with open('Opioid_Drug_List_13_16.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['Excluded'] != 'x':
            if row['Year'] in opiates:
                opiates[row['Year']].append(row['Drug Name'])
            else:
                opiates[row['Year']] = [row['Drug Name']]

From the state-wide data, I used the [Erie and Niagara counties ZIP codes](https://data.ny.gov/Government-Finance/New-York-State-ZIP-Codes-County-FIPS-Cross-Referen/juva-r6g2/data) to filter out none-local results from each dataset.

In [146]:
needed_zips = []
with open('New_York_State_ZIP_Codes-County_FIPS_Cross-Reference.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        needed_zips.append(row['ZIP Code'])
len(needed_zips)

92

In [186]:
payment_2016 = agate.TypeTester(limit=100,force={
        'Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value': agate.Text(),
        'Product_Category_or_Therapeutic_Area_1': agate.Text(),
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1': agate.Text(),
        'Associated_Drug_or_Biological_NDC_1': agate.Text(),
        'Covered_or_Noncovered_Indicator_1': agate.Text(),
        'Product_Category_or_Therapeutic_Area_2': agate.Text(),
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2': agate.Text(),
        'Associated_Drug_or_Biological_NDC_2': agate.Text(),
        'Covered_or_Noncovered_Indicator_2': agate.Text(),
        'Product_Category_or_Therapeutic_Area_3': agate.Text(),
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3': agate.Text(),
        'Associated_Drug_or_Biological_NDC_3': agate.Text(),
        'Covered_or_Noncovered_Indicator_3': agate.Text(),
        'Product_Category_or_Therapeutic_Area_4': agate.Text(),
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4': agate.Text(),
        'Associated_Drug_or_Biological_NDC_4': agate.Text(),
        'Covered_or_Noncovered_Indicator_4': agate.Text(),
        'Product_Category_or_Therapeutic_Area_5': agate.Text(),
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5': agate.Text(),
        'Associated_Drug_or_Biological_NDC_5': agate.Text(),
        'Covered_or_Noncovered_Indicator_5': agate.Text(),
        'Number_of_Payments_Included_in_Total_Amount': agate.Number(),
        'Physician_License_State_code3': agate.Text(),
        'Physician_License_State_code4': agate.Text(),
        'Physician_License_State_code5': agate.Text(),
        'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID': agate.Text(),
        'Physician_Profile_ID': agate.Text()
    })
payment_2015 = agate.TypeTester(limit=100,force={
        'Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value': agate.Text(),
        'Name_of_Associated_Covered_Drug_or_Biological1': agate.Text(),
        'Name_of_Associated_Covered_Drug_or_Biological2': agate.Text(),
        'Name_of_Associated_Covered_Drug_or_Biological3': agate.Text(),
        'Name_of_Associated_Covered_Drug_or_Biological4': agate.Text(),
        'Name_of_Associated_Covered_Drug_or_Biological5': agate.Text(),
        'Number_of_Payments_Included_in_Total_Amount': agate.Number(),
        'NDC_of_Associated_Covered_Drug_or_Biological1': agate.Text(),
        'NDC_of_Associated_Covered_Drug_or_Biological2': agate.Text(),
        'NDC_of_Associated_Covered_Drug_or_Biological3': agate.Text(),
        'NDC_of_Associated_Covered_Drug_or_Biological4': agate.Text(),
        'NDC_of_Associated_Covered_Drug_or_Biological5': agate.Text(),
        'Physician_License_State_code3': agate.Text(),
        'Physician_License_State_code4': agate.Text(),
        'Physician_License_State_code5': agate.Text(),
        'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID': agate.Text(),
        'Physician_Profile_ID': agate.Text(),
        'Teaching_Hospital_CCN': agate.Text(),
        'Teaching_Hospital_ID': agate.Text(),
        'Teaching_Hospital_Name': agate.Text(),
        'Name_of_Associated_Covered_Device_or_Medical_Supply1': agate.Text(),
        'Name_of_Associated_Covered_Device_or_Medical_Supply2': agate.Text(),
        'Name_of_Associated_Covered_Device_or_Medical_Supply3': agate.Text(),
        'Name_of_Associated_Covered_Device_or_Medical_Supply4': agate.Text(),
        'Name_of_Associated_Covered_Device_or_Medical_Supply5': agate.Text(),
    })
scripts_tester = agate.TypeTester(limit=100, force={
        'beneficiary_race_nat_ind_count': agate.Number(),
        'nppes_provider_zip5': agate.Text(),
        'nppes_provider_zip4': agate.Text(),
        'npi': agate.Text()
    })

This program takes in the csv of a specific year's assesment data and its tester and returns a dictionary of BEDS (state id codes) and test data.

In [187]:
docs = {}

In [198]:
def merge_year_data(year):
    scripts_file = 'Medicare_Perscriber_Summary_{0}.csv'.format(year)
    print('Reading in {0}'.format(scripts_file))
    scripts = agate.Table.from_csv(scripts_file, column_types=scripts_tester)
    payments_file = 'General_Payment_NY_{0}.csv'.format(year)
    print('Reading in {0}'.format(payments_file))
    old_data_format = int(year) < 2016
    if old_data_format:
        payments = agate.Table.from_csv(payments_file, column_types=payment_2015)
    else:
        payments = agate.Table.from_csv(payments_file, column_types=payment_2016)
    def check_zip(zip_code):
        if '-' in zip_code:
            zip_list = zip_code.split('-')
            if zip_list[0] in needed_zips:
                return True
            else:
                return False
        else:
            if zip_code in needed_zips:
                return True
            else:
                return False
    local_payments = payments.where(lambda row: check_zip(row['Recipient_Zip_Code']))
    print('Out of {0} payments, {1} are local: {2} percent'.format(len(payments), len(local_payments), (len(local_payments)/len(payments))))
    local_scripts = scripts.where(lambda row: check_zip(row['nppes_provider_zip5']))
    print('Out of {0} perscribers, {1} are local: {2} percent'.format(len(scripts), len(local_scripts), (len(local_scripts)/len(scripts))))
    def opiate_test(row):
        if old_data_format:
            if row['Name_of_Associated_Covered_Drug_or_Biological1'] != None:
                upper1 = row['Name_of_Associated_Covered_Drug_or_Biological1'].upper()
                if upper1 in opiates[year]:
                    return True
            elif row['Name_of_Associated_Covered_Drug_or_Biological2'] != None:
                upper2 = row['Name_of_Associated_Covered_Drug_or_Biological2'].upper()
                if upper2 in opiates[year]:
                    return True
            elif row['Name_of_Associated_Covered_Drug_or_Biological3'] != None:
                upper3 = row['Name_of_Associated_Covered_Drug_or_Biological3'].upper()
                if upper3 in opiates[year]:
                    return True
            elif row['Name_of_Associated_Covered_Drug_or_Biological4'] != None:
                upper4 = row['Name_of_Associated_Covered_Drug_or_Biological4'].upper()
                if upper4 in opiates[year]:
                    return True
            elif row['Name_of_Associated_Covered_Drug_or_Biological5'] != None:
                upper5 = row['Name_of_Associated_Covered_Drug_or_Biological5'].upper()
                if upper5 in opiates[year]:
                    return True
            else:
                return False  
        else:
            if row['Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1'] == 'Drug':
                if row['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1'] != None:
                    upper1 = row['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1'].upper()
                    if upper1 in opiates[year]:
                        return True
            elif row['Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2'] == 'Drug':
                if row['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2'] != None:
                    upper2 = row['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2'].upper()
                    if upper2 in opiates[year]:
                        return True
            elif row['Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3'] == 'Drug':
                if row['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3'] != None:
                    upper3 = row['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3'].upper()
                    if upper3 in opiates[year]:
                        return True
            elif row['Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4'] == 'Drug':
                if row['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4'] != None:
                    upper4 = row['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4'].upper()
                    print(upper4)
                    if upper4 in opiates[year]:
                        print('matched')
                        return True
            else:
                return False
    local_opiate_payments = local_payments.compute([
        ('opiate_flag', agate.Formula(agate.Boolean(), lambda row: opiate_test(row)))
    ]).where(lambda row: row['opiate_flag'] == True)
    if old_data_format:
        print('Out of {0} local payments and {2} drug related, {1} were opioid-related.'.format(len(local_payments),len(local_opiate_payments), len(local_payments.where(lambda row: row['Name_of_Associated_Covered_Drug_or_Biological1'] != None))))
    else:
        print('Out of {0} local payments and {2} drug related, {1} were opioid-related.'.format(len(local_payments),len(local_opiate_payments), len(local_payments.where(lambda row: row['Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1'] == 'Drug'))))
    for payment in local_opiate_payments.rows:
        doc_id = payment['Physician_Profile_ID']
        if doc_id in docs:
            if year in docs[doc_id]:
                old_total = docs[doc_id][year]['total']
                old_count = docs[doc_id][year]['count']
                old_total += payment['Total_Amount_of_Payment_USDollars']
                old_count += payment['Number_of_Payments_Included_in_Total_Amount']
                docs[doc_id][year]['total'] = old_total
                docs[doc_id][year]['count'] = old_count
            else:
                docs[doc_id][year] = {'total': payment['Total_Amount_of_Payment_USDollars'], 'count': payment['Number_of_Payments_Included_in_Total_Amount']}
        else:
            docs[doc_id] = {'info': {'last_name': payment['Physician_Last_Name'], 'first_name': payment['Physician_First_Name'], 'middle_name': payment['Physician_Middle_Name'], 'zip_code': payment['Recipient_Zip_Code'], 'city': payment['Recipient_City'] }, year: {'total':payment['Total_Amount_of_Payment_USDollars'], 'count': payment['Number_of_Payments_Included_in_Total_Amount'] }}
    matched_full = 0
    matched_part = 0
    potential_matches = 0
    for doc_id, id_info in docs.items():
        matched = False
        if 'npi' in docs[doc_id]['info']:
            matched = local_scripts.where(lambda row: row['npi'] == docs[doc_id]['info']['npi'])
            print('####len of matched given current npi###')
            print(len(matched))
            print(docs[doc_id])
            if len(matched) == 1:
                docs[doc_id][year]['opioid_claim_count'] = matched[0]['opioid_claim_count']
                docs[doc_id][year]['opioid_drug_cost'] = matched[0]['opioid_drug_cost']
                docs[doc_id][year]['opioid_day_supply'] = matched[0]['opioid_day_supply']
                docs[doc_id][year]['opioid_bene_count'] = matched[0]['opioid_bene_count']
                docs[doc_id][year]['opioid_prescriber_rate'] = matched[0]['opioid_prescriber_rate']
                print(matched[0]['opioid_claim_count'])
        for doc in local_scripts.rows:
            if '-' in id_info['info']['zip_code']:
                clean_zip = id_info['info']['zip_code'].split('-')[0]
            else:
                clean_zip = id_info['info']['zip_code']
            if id_info['info']['last_name'] == doc['nppes_provider_last_org_name'] and id_info['info']['first_name'] == doc['nppes_provider_first_name'] and id_info['info']['middle_name'] == doc['nppes_provider_mi'] and clean_zip == doc['nppes_provider_zip5']:
                matched_full += 1
                matched = True
                docs[doc_id]['info']['npi'] = doc['npi']
                break
            elif id_info['info']['last_name'] == doc['nppes_provider_last_org_name'] and id_info['info']['first_name'] == doc['nppes_provider_first_name'] and clean_zip == doc['nppes_provider_zip5']:
                matched_part += 1
                matched = True
                docs[doc_id]['info']['npi'] = doc['npi']
                docs[doc_id][year]['opioid_claim_count'] = doc['opioid_claim_count']
                docs[doc_id][year]['opioid_drug_cost'] = doc['opioid_drug_cost']
                docs[doc_id][year]['opioid_day_supply'] = doc['opioid_day_supply']
                docs[doc_id][year]['opioid_bene_count'] = doc['opioid_bene_count']
                docs[doc_id][year]['opioid_prescriber_rate'] = doc['opioid_prescriber_rate']
                break
        if not matched:
            same_last = local_scripts.where(lambda row: id_info['info']['last_name'] == row['nppes_provider_last_org_name']).select(['nppes_provider_last_org_name', 'nppes_provider_first_name', 'nppes_provider_mi','nppes_provider_zip5', 'nppes_provider_city'])
            if len(same_last) != 0:
                print('Finding {0}, {1} {2} {3} {4}'.format(id_info['info']['last_name'], id_info['info']['first_name'], id_info['info']['middle_name'],id_info['info']['zip_code'], id_info['info']['city']))
                same_last.print_table()
                potential_matches += 1
    print('****Out of {0} local opiate paymenters found payment info on {1} matches, {2} percent and potential clean on {3}****'.format(len(docs), (matched_full + matched_part), ((matched_full + matched_part)/len(docs)),potential_matches ))     

In [191]:
merge_year_data('2015')

Reading in Medicare_Perscriber_Summary_2015.csv
Reading in General_Payment_NY_2015.csv
Out of 818414 payments, 52247 are local: 0.06383932826173551 percent
Out of 89057 perscribers, 4529 are local: 0.05085507034820396 percent
Out of 52247 local payments and 41470 drug related, 896 were opioid-related.
Finding CANGE, JEAN MARCAISSE None 14221 WILLIAMSVILLE
| nppes_provider_la... | nppes_provider_fi... | nppes_provider_mi | nppes_provider_zip5 | nppes_provider_city |
| -------------------- | -------------------- | ----------------- | ------------------- | ------------------- |
| CANGE                | JEAN                 | M                 | 14221               | WILLIAMSVILLE       |
Finding RAMKUMAR, BHUVANESWARI Guntur 14225 Buffalo
| nppes_provider_la... | nppes_provider_fi... | nppes_provider_mi | nppes_provider_zip5 | nppes_provider_city |
| -------------------- | -------------------- | ----------------- | ------------------- | ------------------- |
| RAMKUMAR             | BHUVA

In [199]:
merge_year_data('2016')

Reading in Medicare_Perscriber_Summary_2016.csv
Reading in General_Payment_NY_2016.csv
Out of 804403 payments, 50596 are local: 0.0628988206160345 percent
Out of 91449 perscribers, 4561 are local: 0.04987479360080482 percent
Out of 50596 local payments and 33982 drug related, 587 were opioid-related.
####len of matched given current npi###
1
81
####len of matched given current npi###
1


KeyError: '2016'

In [163]:
print(docs)

{'315621': {'info': {'middle_name': None, 'city': 'TONAWANDA', 'first_name': 'WILLIAM', 'zip_code': '14150', 'npi': '1952370173', 'last_name': 'STEPHAN'}, '2016': {'count': Decimal('10'), 'opioid_drug_cost': Decimal('1659.33'), 'opioid_claim_count': Decimal('58'), 'total': Decimal('157.80'), 'opioid_bene_count': Decimal('34'), 'opioid_day_supply': Decimal('694'), 'opioid_prescriber_rate': Decimal('0.35')}}, '77696': {'info': {'middle_name': 'E', 'city': 'HAMBURG', 'first_name': 'JESSICA', 'zip_code': '14075-2600', 'npi': '1720221229', 'last_name': 'OGORCHOCK-CONGILOSI'}, '2016': {'count': Decimal('2'), 'total': Decimal('33.52')}}, '161228': {'info': {'middle_name': 'S', 'city': 'DERBY', 'first_name': 'PETER', 'zip_code': '14047-9591', 'npi': '1558326173', 'last_name': 'MCQUILLER'}, '2016': {'count': Decimal('6'), 'total': Decimal('112.26')}}, '3109': {'info': {'middle_name': None, 'city': 'ORCHARD PARK', 'first_name': 'PAUL', 'zip_code': '14127', 'npi': '1073558557', 'last_name': 'BIDD